In [4]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

import fish_models
import robofish.io

In [5]:
fishes = 4

In [13]:
from sklearn.mixture import GaussianMixture
import sklearn.cluster as cl
import sklearn.metrics as mt

class EMAlgorithmFishModel(fish_models.gym_interface.AbstractRaycastBasedModel):
    """
    Representation of a Gaussian mixture model probability distribution. 
    GMMs are probabilistic models that assume all the data points are generated 
    from a mixture of several Gaussian distributions with unknown parameters.

    The EM algorithm is an iterative approach that cycles between two modes:
    E-Step. Estimate the missing variables in the dataset.
    M-Step. Maximize the parameters of the model in the presence of the data.
    """
    
    def choose_action(self, view: np.ndarray):
        
        prediction = self.model.predict([view])
        
        print(prediction)
        print(self.model.get_params())
        
        speed = prediction[0]
        turn = prediction[1]
        
        # turn correction for walls avoidance
        turn = self.avoid_walls(view, turn)
        
        return speed, turn
    
    def avoid_walls(self, view, turn):
        """
        Forces to turn a fish in a random direction
        if in a view's raycast of the walls
        a wall in the front of a fish is detected to near

        Parameters
        ---------
        view : array_like
            The observations of the virtual fish
        turn : float
            Turn predicted by a model that is to modify

        Returns
        ---------
        turn : float
            Original or modified turn depending on the wall distance
        """
        param = random.randint(-5, 5)
        
        if param == 0:
            param = random.randint(5, 11)
        
        if view[6] > 0.9:
            return param * np.pi
        else:
            return turn
        
    def learn_params(self, x_labeled, y_labeled):
        n = x_labeled.shape[0]
        phi = x_labeled[y_labeled == 1].shape[0] / n
        mu0 = np.sum(x_labeled[y_labeled == 0], axis=0) / x_labeled[y_labeled == 0].shape[0]
        mu1 = np.sum(x_labeled[y_labeled == 1], axis=0) / x_labeled[y_labeled == 1].shape[0]
        sigma0 = np.cov(x_labeled[y_labeled == 0].T, bias= True)
        sigma1 = np.cov(x_labeled[y_labeled == 1].T, bias=True)
        return {'phi': phi, 'mu0': mu0, 'mu1': mu1, 'sigma0': sigma0, 'sigma1': sigma1}

    def fit(self, dset):
        """
        Learns parameters and runs the the two steps of EM Algorithm 
        until the average log-likelihood converges.
        """
        
        X = dset[:]["views"]
        y = dset[:]["actions"]
        
        
        learned_params = self.learn_params(X, y)
        weights = [1 - learned_params["phi"], learned_params["phi"]]
        means = [learned_params["mu0"], learned_params["mu1"]]
        covariances = [learned_params["sigma0"], learned_params["sigma1"]]
        
        self.model = GaussianMixture(n_components=2,
                                     covariance_type='full',
                                     tol=0.01,
                                     max_iter=1000,
                                     weights_init=weights,
                                     means_init=means,
                                     precisions_init=covariances)
        
        
        self.model.fit(X, y)
        
        print("\nscikit learn:\n\tphi: %s\n\tmu_0: %s\n\tmu_1: %s\n\tsigma_0: %s\n\tsigma_1: %s"
               % (model.weights_[1], model.means_[0, :], model.means_[1, :], model.covariances_[0, :], model.covariances_[1, :]))
     
        
    
model = EMAlgorithmFishModel()

In [14]:
raycast = fish_models.gym_interface.Raycast(
            n_wall_raycasts=5,
            n_fish_bins=4,
            fov_angle_fish_bins=np.pi,
            fov_angle_wall_raycasts=np.pi,
            world_bounds=([-50, -50], [50, 50]),
        )



In [8]:
data_folder = Path("data/live_female_female/train")

dset = fish_models.datasets.io_dataset.IoDataset(
    data_folder,
    raycast,
    output_strings=["poses", "actions", "views"],
    reduce_dim=2,
    max_files=5,
)

  0%|          | 0/5 [00:00<?, ?it/s]

Loading data from 5 files.


  0%|          | 0/5 [00:00<?, ?it/s]

Calculating views from 5 files.


100%|██████████| 5/5 [00:09<00:00,  1.84s/it]

Created IoDataset:
Reduced the first 3 dimensions from (5, 2, 8989) to (89890)
poses	(89890, 3):	consisting of x, y, calc_ori_rad.
actions	(89880, 2):	consisting of speed[cm/s] and turn [rad/s].
views	(89880, 9):	4 fish_bins and 5 wall ray casts.



In [15]:
model.fit(dset)

IndexError: boolean index did not match indexed array along dimension 1; dimension is 9 but corresponding boolean dimension is 2

In [ ]:
generator = fish_models.gym_interface.TrackGeneratorGymRaycast(
    model, raycast, [100,100], 25
)

track = generator.create_track(n_guppies=fishes, trackset_len=5000)

In [ ]:
f = generator.as_io_file(track)
f.save_as("output/em_algorithm.hdf5")

In [ ]:
plt.figure(figsize=(10,10))
plt.xlim(-50,50)
plt.ylim(-50,50)
for fish_id in range(fishes):
    plt.plot(track[fish_id, :, 0], track[fish_id, :, 1])
plt.show()